# Chatbot pour l'Université d'Abomey-Calavi (UAC)

Ce notebook implémente un chatbot pour l'Université d'Abomey-Calavi en utilisant des techniques de traitement du langage naturel et de recherche d'information.

## Installation des dépendances

Commençons par installer les bibliothèques nécessaires.

In [1]:
%pip install -q qdrant-client spacy langchain langchain-google-genai langchain_cohere

Note: you may need to restart the kernel to use updated packages.


## Importation des bibliothèques

Importons maintenant toutes les bibliothèques et modules nécessaires.

In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_cohere import CohereEmbeddings
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import Qdrant
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import os

USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/hounfodji/Music/uac-gpt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Initialisation du modèle de langage (LLM)

Nous utiliserons le modèle Gemini de Google pour notre chatbot.

In [ ]:
os.environ["GOOGLE_API_KEY"] = "your_api_key"
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

## Collecte de données

Nous allons récupérer des informations à partir du site web de l'UAC.

In [ ]:
urls = ["https://uac.bj/", "https://uac.bj/formation/offre-de-formation/licence/", 
        "https://uac.bj/formation/offre-de-formation/master/", 
        "https://uac.bj/formation/offre-de-formation/doctorat/", 
        "https://uac.bj/decouvrir-luniversite/vision-mission/",
        "https://uac.bj/opportunites-evenements/"]

loader = WebBaseLoader(urls)
docs = loader.load()

## Préparation des données

Nous allons diviser les documents en morceaux plus petits pour un traitement plus efficace.

In [ ]:
!python -m spacy download fr_core_news_sm
splitter = SpacyTextSplitter(pipeline="fr_core_news_sm", chunk_size=2048, chunk_overlap=256)
chunks = splitter.split_documents(docs)

## Création d'embeddings et stockage dans une base de données vectorielle

Nous utiliserons Cohere pour les embeddings et Qdrant comme base de données vectorielle.

In [ ]:
embeddings = CohereEmbeddings(cohere_api_key="your_cohere_api_key")
qdrant = Qdrant.from_documents(
    chunks,
    embeddings,
    path="./local_qdrant",
    collection_name="my_docs",
)

## Configuration du système de récupération

Nous allons configurer notre système pour récupérer les documents pertinents.

In [ ]:
retriever = qdrant.as_retriever(search_kwargs={"k": 12})

system_prompt_rag = (
    "You're a chatbot for University of Abomey-Calavi. "
    "You should give concise answer following user query. "
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Context: {context}"
)

prompt_rag = ChatPromptTemplate.from_messages([
    ("system", system_prompt_rag),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, prompt_rag)
chain = create_retrieval_chain(retriever, question_answer_chain)

## Ajout d'un système de reclassement

Pour améliorer la pertinence des résultats, nous ajoutons un système de reclassement avec Cohere.

In [ ]:
compressor = CohereRerank(cohere_api_key="34nAQRvTUN8OPvWp22gaE7WvkQbDQoDfbk81knz8", top_n=10)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)
reranker_chain = create_retrieval_chain(compression_retriever, question_answer_chain)

## Fonction chatbot

Enfin, nous créons une fonction simple pour interagir avec notre chatbot.

In [ ]:
def chatbot(query: str):
    result = reranker_chain.invoke({"input": query})
    return result["answer"]

# Exemple d'utilisation
response = chatbot("Qui est le recteur de l'UAC ?")
print(response)